In [10]:
import numpy as np
import torch
import torch.nn as nn
import random
import os

from sklearn.datasets import make_s_curve
import matplotlib.pyplot as plt

from models.ddpm.net import MLPDiffusion


device = torch.device('cpu')

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

# model = MLPDiffusion(
#     s_dim=2,
#     hidden_dim=256,
#     hidden_layers=3,
#     n_steps=100,
# ).to(device)
#
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print (name, param.data.shape)

class DDPM(nn.Module):

    def __init__(
        self,
        s_dim,
        diff_steps,
        hidden_dim=128,
        hidden_layers=3,
    ):
        super().__init__()

        self.s_dim = s_dim
        self.diff_steps = diff_steps

        self.noise_net = MLPDiffusion(
            s_dim=self.s_dim,
            hidden_dim=hidden_dim,
            hidden_layers=hidden_layers,
            n_steps=self.diff_steps,
        )

        






model = DDPM(
    s_dim=2,
    diff_steps=100,
)

for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data.shape)






noise_net.net.0.weight torch.Size([128, 2])
noise_net.net.0.bias torch.Size([128])
noise_net.net.2.weight torch.Size([128, 128])
noise_net.net.2.bias torch.Size([128])
noise_net.net.4.weight torch.Size([128, 128])
noise_net.net.4.bias torch.Size([128])
noise_net.net.6.weight torch.Size([2, 128])
noise_net.net.6.bias torch.Size([2])
noise_net.t_embeds.0.weight torch.Size([100, 128])
noise_net.t_embeds.1.weight torch.Size([100, 128])
noise_net.t_embeds.2.weight torch.Size([100, 128])
